In [1]:
import sys
sys.path.append("..")
sys.path.append("../../eqnet")

In [2]:
from expemb import TestingResults, TrainingArguments, EquivExpTokenizer
import glob
import gzip
import os
import csv
import json
from sympy import latex

/projects/kani-lab/user/gangwar2/.miniconda3/envs/expembtx/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Equivalent Expression Generation

### Test Set

In [3]:
expembe_model = "../models/equivexp/20221126-104533874570"
expemba_model = "../models/equivexp/20221127-082715223727"

In [4]:
def read_results(modeldir, autoencoder, testfile_prefix):
    trainargsfile = os.path.join(modeldir, "train_args.yaml")
    testfile_template = os.path.join(modeldir, f"{testfile_prefix}*.yaml")
    filelist = glob.glob(testfile_template)
    assert len(filelist) == 1
    filepath = filelist[0]
    args = TrainingArguments.load(trainargsfile)
    if autoencoder:
        assert args.autoencoder
    else:
        assert not args.autoencoder
    
    results = TestingResults.load(filepath)
    return results.accuracy[0]


def print_eqexp_test_results_table():
    expembe_results = read_results(expembe_model, False, "test")
    expemba_results = read_results(expemba_model, True, "test")
    
    print("\\toprule")
    print("Beam Size & \\expemba{} & \\expembe{} \\\\")
    print("\\midrule")
    print(f"1 & {expemba_results['test/accuracy_1']:.4f} & {expembe_results['test/accuracy_1']:.4f} \\\\")
    print(f"10 & {expemba_results['test/accuracy_10']:.4f} & {expembe_results['test/accuracy_10']:.4f} \\\\")
    print(f"50 & {expemba_results['test/accuracy_50']:.4f} & {expembe_results['test/accuracy_50']:.4f} \\\\")
    print("\\bottomrule")
    
print_eqexp_test_results_table()

\toprule
Beam Size & \expemba{} & \expembe{} \\
\midrule
1 & 0.9996 & 0.7206 \\
10 & 0.9998 & 0.8118 \\
50 & 0.9998 & 0.8386 \\
\bottomrule


### Validation Set

In [5]:
def print_eqexp_validation_results_table():
    expembe_results = read_results(expembe_model, False, "validation")
    expemba_results = read_results(expemba_model, True, "validation")
    
    print("\\toprule")
    print("Beam Size & \\expemba{} & \\expembe{} \\\\")
    print("\\midrule")
    print(f"1 & {expemba_results['test/accuracy_1']:.4f} & {expembe_results['test/accuracy_1']:.4f} \\\\")
    print(f"10 & {expemba_results['test/accuracy_10']:.4f} & {expembe_results['test/accuracy_10']:.4f} \\\\")
    print(f"50 & {expemba_results['test/accuracy_50']:.4f} & {expembe_results['test/accuracy_50']:.4f} \\\\")
    print("\\bottomrule")
    
print_eqexp_validation_results_table()

\toprule
Beam Size & \expemba{} & \expembe{} \\
\midrule
1 & 0.9995 & 0.7615 \\
10 & 0.9995 & 0.8365 \\
50 & 0.9995 & 0.8665 \\
\bottomrule


### Analogy

In [6]:
def read_analogy_results(model):
    resultfiletemplate = os.path.join(model, "emb_math_results*.csv")
    resultfilelist = glob.glob(resultfiletemplate)
    assert len(resultfilelist) == 1, f"Incorrect number of files for {resultfiletemplate}"
    
    resultfile = resultfilelist[0]
    results = []
    with open(resultfile, "r") as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
            results.append({
                "x1": row["x1"],
                "y1": row["y1"],
                "y2": row["y2"],
                "expected_x2": row["expected_x2"],
                "predicted_x2": row["predicted_x2"],
            })
            
    return results


def print_analogy_results_table():
    expemba_results = read_analogy_results(expemba_model)
    expembe_results = read_analogy_results(expembe_model)
    
    tokenizer = EquivExpTokenizer()
    
    assert len(expemba_results) == len(expembe_results)
    
    print("\\toprule")
    print("\\multirow{2}{*}{$x_1$} & \\multirow{2}{*}{$y_1$} & \\multirow{2}{*}{$y_2$} & \\multirow{2}{*}{$x_2$ (expected)} & \\multicolumn{2}{c}{$x_2$ (predicted)} \\\\")
    print("{} & {} & {} & {} & \\expemba{} & \\expembe{} \\\\")
    print("\\midrule")
    
    for idx in range(len(expembe_results)):
        expembe_result = expembe_results[idx]
        expemba_result = expemba_results[idx]
        
        assert expembe_result["x1"] == expemba_result["x1"]
        assert expembe_result["y1"] == expemba_result["y1"]
        assert expembe_result["y2"] == expemba_result["y2"]
        assert expembe_result["expected_x2"] == expemba_result["expected_x2"]
        
        x1 = tokenizer.prefix_to_sympy(expembe_result["x1"], evaluate = False)
        y1 = tokenizer.prefix_to_sympy(expembe_result["y1"], evaluate = False)
        y2 = tokenizer.prefix_to_sympy(expembe_result["y2"], evaluate = False)
        expected_x2 = tokenizer.prefix_to_sympy(expembe_result["expected_x2"], evaluate = False)
        expembe_x2 = tokenizer.prefix_to_sympy(expembe_result["predicted_x2"], evaluate = False)
        expemba_x2 = tokenizer.prefix_to_sympy(expemba_result["predicted_x2"], evaluate = False)
        
        expembe_bold = "\\boldmath" if expembe_result["expected_x2"] == expembe_result["predicted_x2"] else ""
        expemba_bold = "\\boldmath" if expemba_result["expected_x2"] == expemba_result["predicted_x2"] else ""
        print(
            f"${latex(x1)}$ & ${latex(y1)}$ & ${latex(y2)}$ & ${latex(expected_x2)}$ & " + \
            f"{expemba_bold} ${latex(expemba_x2)}$ & {expembe_bold} ${latex(expembe_x2)}$ \\\\"
        )
        
    print("\\bottomrule")


print_analogy_results_table()

\toprule
\multirow{2}{*}{$x_1$} & \multirow{2}{*}{$y_1$} & \multirow{2}{*}{$y_2$} & \multirow{2}{*}{$x_2$ (expected)} & \multicolumn{2}{c}{$x_2$ (predicted)} \\
{} & {} & {} & {} & \expemba{} & \expembe{} \\
\midrule
$\cos{\left(x \right)}$ & $\sin{\left(x \right)}$ & $\csc{\left(x \right)}$ & $\sec{\left(x \right)}$ & \boldmath $\sec{\left(x \right)}$ & \boldmath $\sec{\left(x \right)}$ \\
$\cos{\left(x \right)}$ & $\sin{\left(x \right)}$ & $\cot{\left(x \right)}$ & $\tan{\left(x \right)}$ &  $\sqrt{x}$ &  $x + \cot{\left(x \right)}$ \\
$\sin{\left(x \right)}$ & $\cos{\left(x \right)}$ & $\cosh{\left(x \right)}$ & $\sinh{\left(x \right)}$ &  $\tanh{\left(x \right)}$ & \boldmath $\sinh{\left(x \right)}$ \\
$\sin{\left(x \right)}$ & $\csc{\left(x \right)}$ & $\sec{\left(x \right)}$ & $\cos{\left(x \right)}$ & \boldmath $\cos{\left(x \right)}$ & \boldmath $\cos{\left(x \right)}$ \\
$\sin{\left(x \right)}$ & $\csc{\left(x \right)}$ & $\cot{\left(x \right)}$ & $\tan{\left(x \right)}$ &  $\

### Distance Analysis

In [7]:
def read_dist_analysis_results(modeldir, autoencoder):
    trainargsfile = os.path.join(modeldir, "train_args.yaml")
    testfile_template = os.path.join(modeldir, f"dist_analysis_results*.json")
    filelist = glob.glob(testfile_template)
    assert len(filelist) == 1
    filepath = filelist[0]
    args = TrainingArguments.load(trainargsfile)
    if autoencoder:
        assert args.autoencoder
    else:
        assert not args.autoencoder
    
    with open(filepath, "r") as f:
        results = json.load(f) 
    return results


def print_dist_analysis_table():
    expembe_results = read_dist_analysis_results(expembe_model, False)
    expemba_results = read_dist_analysis_results(expemba_model, True)
    
    tokenizer = EquivExpTokenizer()
    queries = list(expembe_results.keys())
    
    print("\\toprule")
    print("Query & \\expemba{} & \\expembe{} \\\\")
    for query in queries:
        e_nns = expembe_results[query]
        a_nns = expemba_results[query]
        
        print("\n\\midrule")
        query = tokenizer.prefix_to_sympy(query)
        for idx, (e_nn, a_nn) in enumerate(zip(e_nns, a_nns)):            
            e_nn = tokenizer.prefix_to_sympy(e_nn)
            a_nn = tokenizer.prefix_to_sympy(a_nn)
            
            if idx == 0:
                print(
                    "\\multirow{5}{*}{" + f"${latex(query)}$ " + "} & " + \
                    f"${latex(a_nn)}$ & ${latex(e_nn)}$ \\\\"
                )
            else:
                print(f" & ${latex(a_nn)}$ & ${latex(e_nn)}$ \\\\")
        
    print("\\bottomrule")
    
print_dist_analysis_table()

\toprule
Query & \expemba{} & \expembe{} \\

\midrule
\multirow{5}{*}{$4 x^{2} \cos{\left(3 x - 1 \right)}$ } & $4 x^{2} e^{- 3 x - 1}$ & $- 10 x^{2} \cos{\left(x - 10 \right)}$ \\
 & $4 x^{2} e^{2 x - 1}$ & $- 10 x^{2} \cos{\left(x + 10 \right)}$ \\
 & $4 x^{2} e^{- 2 x - 1}$ & $x^{2} \cos{\left(x - 10 \right)}$ \\
 & $4 x^{2} e^{- 3 x - 9}$ & $x^{2} \cos{\left(x + 1 \right)}$ \\
 & $4 x^{2} e^{- x - 1}$ & $x^{2} \cos{\left(x - 30 \right)}$ \\

\midrule
\multirow{5}{*}{$21 x - 3 \sin{\left(x \right)}$ } & $- x \log{\left(x \right)} + 21 x$ & $210 x + 210 \sin{\left(x \right)}$ \\
 & $- 3 x \operatorname{acosh}{\left(x \right)} + 21 x$ & $- 60 x + 20 \sin{\left(x \right)}$ \\
 & $2 x - \frac{16 \log{\left(x \right)}}{3}$ & $11 x + 11 \sin{\left(x \right)}$ \\
 & $- 10 x \cos{\left(x \right)} + 21 x$ & $21 x + 21 \cos{\left(x \right)}$ \\
 & $- x e^{x} + 21 x$ & $- 70 x + 10 \sin{\left(x \right)}$ \\

\midrule
\multirow{5}{*}{$x^{2} + \log{\left(x \right)} + 1$ } & $\sqrt{x} + x^{2} + \

## SemVec Results

### EqNet and EqNet-L Results

In [8]:
datasets = {
    "boolean8": "\\textsc{Bool8}",
    "oneVarPoly13": "\\textsc{oneV-Poly13}",
    "simplepoly10": "\\textsc{SimpPoly10}",
    "simpleBoolean8": "\\textsc{SimpBool8}",
    "boolean10": "\\textsc{Bool10}",
    "simpleBoolean10": "\\textsc{SimpBool10}",
    "largeBoolean5": "\\textsc{BoolL5}",
    "poly8": "\\textsc{Poly8}",
    "simplepoly8": "\\textsc{SimpPoly8}",
    "largeSimpleBoolean5": "\\textsc{SimpBoolL5}",
    "oneVarPoly10": "\\textsc{oneV-Poly10}",
    "boolean5": "\\textsc{Bool5}",
    "poly5": "\\textsc{Poly5}",
    "simplepoly5": "\\textsc{SimpPoly5}",
}
all_hidden_sizes = [64, 128, 256, 512, 1024]
eqnet_results = {
    "simpleBoolean8": "97.4",
    "simpleBoolean10": "99.1",
    "boolean5": "65.8",
    "boolean8": "58.1",
    "boolean10": "71.4",
    "largeSimpleBoolean5": "85.0",
    "largeBoolean5": "75.2",
    "simplepoly5": "65.6",
    "simplepoly8": "98.9",
    "simplepoly10": "99.3",
    "oneVarPoly10": "81.3",
    "oneVarPoly13": "90.4",
    "poly5": "55.3",
    "poly8": "86.2",
}
eqnetl_results = {
    "simpleBoolean8": "-",
    "simpleBoolean10": "-",
    "boolean5": "73.7",
    "boolean8": "-",
    "boolean10": "-",
    "largeSimpleBoolean5": "72.1",
    "largeBoolean5": "-",
    "simplepoly5": "56.3",
    "simplepoly8": "98.0",
    "simplepoly10": "-",
    "oneVarPoly10": "80.0",
    "oneVarPoly13": "-",
    "poly5": "-",
    "poly8": "87.1",
}

In [9]:
def read_expemb_results(dataset, autoencoder, hidden = 64):
    if autoencoder:
        resultfilepattern = f"../models/semvec/autoencoder/h{hidden}/{dataset}/results*"
    else:
        resultfilepattern = f"../models/semvec/equivexp/{dataset}/results*"
    
    resultfilelist = glob.glob(resultfilepattern)
    assert len(resultfilelist) == 1
    resultfile = resultfilelist[0]
    
    results = TestingResults.load(resultfile)
    
    # Validate the test files
    assert results.args.test_file.endswith(f"{dataset}-neweqtestset.json.gz"), f"Invalid test file {results.args.test_file}"
    assert results.args.full_file.endswith(f"{dataset}.json.gz"), f"Invalid full file {results.args.full_file}"
    
    return results.accuracy[0]["val/score@5/max"]


def get_training_set_size(dataset, autoencoder):
    if autoencoder:
        trainfile = f"../data/semvec/{dataset}_autoenc.train.gz"
    else:
        trainfile = f"../data/semvec/{dataset}.train.gz"
        
    with gzip.open(trainfile, "rt") as file:
        n_lines = len(file.readlines())
        
    return n_lines

### Main Results Table

In [10]:
def print_main_results_table():
    scores = []

    for dataset, displayname in datasets.items():
        eqnet_score = eqnet_results[dataset]
        eqnetl_score = eqnetl_results[dataset]
        expembe_score = float(read_expemb_results(dataset, autoencoder = False))
        expemba_score = float(read_expemb_results(dataset, autoencoder = True))
        expembe_train_size = get_training_set_size(dataset, autoencoder = False)
        expemba_train_size = get_training_set_size(dataset, autoencoder = True)
        
        expemba_score = round(expemba_score * 100, 1)
        expembe_score = round(expembe_score * 100, 1)

        scores.append({
            "dataset": displayname,
            "eqnet": eqnet_score,
            "eqnetl": eqnetl_score,
            "expemba": expemba_score,
            "expemba_train_size": expemba_train_size,
            "expembe": expembe_score,
            "expembe_train_size": expembe_train_size,
        })
        
    # Sort based on ExpEmb-E training set size
    scores = sorted(scores, key = lambda d : int(d["expembe_train_size"]), reverse = True)

    print("\\toprule")
    print("\\multirow{2}{*}{Dataset} & \\eqnet{} & \\eqnetl{} & \\multicolumn{2}{c}{\\expemba{}} & \\multicolumn{2}{c}{\\expembe{}} \\\\")
    print("& $score_5(\\%)$ & $score_5(\\%)$ & $score_5(\\%)$ & Training Set Size & $score_5(\\%)$ & Training Set Size \\\\")
    print("\\midrule")
    for score in scores:
        print(
            f"{score['dataset']} & {score['eqnet']} & {score['eqnetl']} & " + \
            f"{score['expemba']} & {score['expemba_train_size']:,} & " + \
            f"{score['expembe']} & {score['expembe_train_size']:,} \\\\"
        )
        
    print(f"\\bottomrule")
    
print_main_results_table()

\toprule
\multirow{2}{*}{Dataset} & \eqnet{} & \eqnetl{} & \multicolumn{2}{c}{\expemba{}} & \multicolumn{2}{c}{\expembe{}} \\
& $score_5(\%)$ & $score_5(\%)$ & $score_5(\%)$ & Training Set Size & $score_5(\%)$ & Training Set Size \\
\midrule
\textsc{Bool8} & 58.1 & - & 30.6 & 146,488 & 100.0 & 16,143,072 \\
\textsc{oneV-Poly13} & 90.4 & - & 38.7 & 60,128 & 99.6 & 9,958,406 \\
\textsc{SimpPoly10} & 99.3 & - & 40.1 & 31,143 & 99.8 & 6,731,858 \\
\textsc{SimpBool8} & 97.4 & - & 36.9 & 21,604 & 99.4 & 4,440,450 \\
\textsc{Bool10} & 71.4 & - & 10.8 & 25,560 & 91.3 & 3,041,640 \\
\textsc{SimpBool10} & 99.1 & - & 24.4 & 13,081 & 95.5 & 1,448,804 \\
\textsc{BoolL5} & 75.2 & - & 38.3 & 23,219 & 36.0 & 552,642 \\
\textsc{Poly8} & 86.2 & 87.1 & 32.7 & 6,785 & 87.3 & 257,190 \\
\textsc{SimpPoly8} & 98.9 & 98.0 & 47.6 & 1,934 & 98.9 & 113,660 \\
\textsc{SimpBoolL5} & 85.0 & 72.1 & 55.1 & 6,009 & 71.1 & 66,876 \\
\textsc{oneV-Poly10} & 81.3 & 80.0 & 59.8 & 767 & 74.1 & 25,590 \\
\textsc{Bool5} & 65.

### Autoencoder All Hidden Sizes

In [11]:
def print_expemba_all_results():
    scores = []

    for dataset, displayname in datasets.items():
        h32_score = float(read_expemb_results(dataset, autoencoder = True, hidden = 32))
        h64_score = float(read_expemb_results(dataset, autoencoder = True, hidden = 64))
        h128_score = float(read_expemb_results(dataset, autoencoder = True, hidden = 128))
        
        scores.append({
            "dataset": displayname,
            "h32": round(h32_score * 100, 1),
            "h64": round(h64_score * 100, 1),
            "h128": round(h128_score * 100, 1),
        })
        
    print("\\toprule")
    print("Dataset & $D_{\\text{model}} = 32$ & $D_{\\text{model}} = 64$ & $D_{\\text{model}} = 128$ \\\\")
    print("\\midrule")
    for score in scores:
        print(f"{score['dataset']} & {score['h32']} & {score['h64']} & {score['h128']} \\\\")
        
    print("\\bottomrule")
        
print_expemba_all_results()

\toprule
Dataset & $D_{\text{model}} = 32$ & $D_{\text{model}} = 64$ & $D_{\text{model}} = 128$ \\
\midrule
\textsc{Bool8} & 31.1 & 30.6 & 25.9 \\
\textsc{oneV-Poly13} & 37.7 & 38.7 & 38.4 \\
\textsc{SimpPoly10} & 47.4 & 40.1 & 44.9 \\
\textsc{SimpBool8} & 33.1 & 36.9 & 34.1 \\
\textsc{Bool10} & 10.7 & 10.8 & 8.6 \\
\textsc{SimpBool10} & 20.5 & 24.4 & 25.4 \\
\textsc{BoolL5} & 35.6 & 38.3 & 38.2 \\
\textsc{Poly8} & 32.2 & 32.7 & 32.1 \\
\textsc{SimpPoly8} & 50.4 & 47.6 & 47.3 \\
\textsc{SimpBoolL5} & 55.2 & 55.1 & 54.9 \\
\textsc{oneV-Poly10} & 58.3 & 59.8 & 59.7 \\
\textsc{Bool5} & 36.7 & 36.4 & 28.1 \\
\textsc{Poly5} & 14.9 & 5.7 & 6.6 \\
\textsc{SimpPoly5} & 18.8 & 28.1 & 17.7 \\
\bottomrule


### Validation Results

In [12]:
def read_expemb_val_results(dataset, autoencoder, hidden = 64):
    if autoencoder:
        resultfilepattern = f"../models/semvec/autoencoder/h{hidden}/{dataset}/validation*"
    else:
        resultfilepattern = f"../models/semvec/equivexp/{dataset}/validation*"
    
    resultfilelist = glob.glob(resultfilepattern)
    assert len(resultfilelist) == 1
    resultfile = resultfilelist[0]
    
    results = TestingResults.load(resultfile)
    
    # Validate the test files
    assert results.args.test_file.endswith(f"{dataset}-validationset.json.gz"), f"Invalid test file {results.args.test_file}"
    assert results.args.full_file.endswith(f"{dataset}.json.gz"), f"Invalid full file {results.args.full_file}"
    
    return results.accuracy[0]["val/score@5/max"]

In [13]:
def print_validation_results_table():
    scores = []

    for dataset, displayname in datasets.items():
        expembe_score = float(read_expemb_val_results(dataset, autoencoder = False))
        expemba_h32_score = float(read_expemb_val_results(dataset, autoencoder = True, hidden = 32))
        expemba_h64_score = float(read_expemb_val_results(dataset, autoencoder = True, hidden = 64))
        expemba_h128_score = float(read_expemb_val_results(dataset, autoencoder = True, hidden = 128))
        
        expembe_score = round(expembe_score * 100, 1)
        expemba_h32_score = round(expemba_h32_score * 100, 1)
        expemba_h64_score = round(expemba_h64_score * 100, 1)
        expemba_h128_score = round(expemba_h128_score * 100, 1)

        scores.append({
            "dataset": displayname,
            "expemba_h32": expemba_h32_score,
            "expemba_h64": expemba_h64_score,
            "expemba_h128": expemba_h128_score,
            "expembe": expembe_score,
        })

    print("\\toprule")
    print("\\multirow{2}{*}{Dataset} & \\multicolumn{3}{c}{\\expemba{}} & \\multirow{2}{*}{\\expembe{}} \\\\")
    print("{} & $D_{\\text{model}} = 32$ & $D_{\\text{model}} = 64$ & $D_{\\text{model}} = 128$ & {} \\\\")
    print("\\midrule")
    for score in scores:
        print(
            f"{score['dataset']} & {score['expemba_h32']} & {score['expemba_h64']} & " + \
            f"{score['expemba_h128']} & {score['expembe']} \\\\"
        )
        
    print(f"\\bottomrule")
    
print_validation_results_table()

\toprule
\multirow{2}{*}{Dataset} & \multicolumn{3}{c}{\expemba{}} & \multirow{2}{*}{\expembe{}} \\
{} & $D_{\text{model}} = 32$ & $D_{\text{model}} = 64$ & $D_{\text{model}} = 128$ & {} \\
\midrule
\textsc{Bool8} & 36.2 & 36.3 & 35.5 & 100.0 \\
\textsc{oneV-Poly13} & 38.3 & 39.0 & 38.5 & 99.9 \\
\textsc{SimpPoly10} & 37.5 & 31.5 & 34.0 & 99.9 \\
\textsc{SimpBool8} & 42.3 & 46.7 & 44.5 & 99.8 \\
\textsc{Bool10} & 9.6 & 9.9 & 9.1 & 93.9 \\
\textsc{SimpBool10} & 23.2 & 26.6 & 27.2 & 96.3 \\
\textsc{BoolL5} & 47.4 & 47.6 & 47.7 & 98.2 \\
\textsc{Poly8} & 36.4 & 36.5 & 36.0 & 98.6 \\
\textsc{SimpPoly8} & 36.5 & 35.4 & 34.6 & 99.8 \\
\textsc{SimpBoolL5} & 75.0 & 76.1 & 74.4 & 99.5 \\
\textsc{oneV-Poly10} & 42.1 & 46.3 & 42.5 & 83.0 \\
\textsc{Bool5} & 42.4 & 42.1 & 35.4 & 81.0 \\
\textsc{Poly5} & 11.6 & 9.5 & 7.4 & 43.2 \\
\textsc{SimpPoly5} & 20.0 & 28.8 & 32.5 & 43.8 \\
\bottomrule


### TestEq (seen classes) Results

In [14]:
def read_expemb_testseeneq_results(dataset, autoencoder, hidden = 64):
    if autoencoder:
        resultfilepattern = f"../models/semvec/autoencoder/h{hidden}/{dataset}/testseeneq-*.yaml"
    else:
        resultfilepattern = f"../models/semvec/equivexp/{dataset}/testseeneq-*.yaml"
    
    resultfilelist = glob.glob(resultfilepattern)
    assert len(resultfilelist) == 1
    resultfile = resultfilelist[0]
    
    results = TestingResults.load(resultfile)
    
    # Validate the test files
    assert results.args.test_file.endswith(f"{dataset}-testset.json.gz"), f"Invalid test file {results.args.test_file}"
    assert results.args.full_file.endswith(f"{dataset}.json.gz"), f"Invalid full file {results.args.full_file}"
    
    return results.accuracy[0]["val/score@5/max"]


def print_testseeneq_results_table():
    scores = []

    for dataset, displayname in datasets.items():
        expembe_score = float(read_expemb_testseeneq_results(dataset, autoencoder = False))
        expemba_h32_score = float(read_expemb_testseeneq_results(dataset, autoencoder = True, hidden = 32))
        expemba_h64_score = float(read_expemb_testseeneq_results(dataset, autoencoder = True, hidden = 64))
        expemba_h128_score = float(read_expemb_testseeneq_results(dataset, autoencoder = True, hidden = 128))
        
        expembe_score = round(expembe_score * 100, 1)
        expemba_h32_score = round(expemba_h32_score * 100, 1)
        expemba_h64_score = round(expemba_h64_score * 100, 1)
        expemba_h128_score = round(expemba_h128_score * 100, 1)

        scores.append({
            "dataset": displayname,
            "expemba_h32": expemba_h32_score,
            "expemba_h64": expemba_h64_score,
            "expemba_h128": expemba_h128_score,
            "expembe": expembe_score,
        })

    print("\\toprule")
    print("\\multirow{2}{*}{Dataset} & \\multicolumn{3}{c}{\\expemba{}} & \\multirow{2}{*}{\\expembe{}} \\\\")
    print("{} & $D_{\\text{model}} = 32$ & $D_{\\text{model}} = 64$ & $D_{\\text{model}} = 128$ & {} \\\\")
    print("\\midrule")
    for score in scores:
        print(
            f"{score['dataset']} & {score['expemba_h32']} & {score['expemba_h64']} & " + \
            f"{score['expemba_h128']} & {score['expembe']} \\\\"
        )
        
    print(f"\\bottomrule")
    
    
print_testseeneq_results_table()

\toprule
\multirow{2}{*}{Dataset} & \multicolumn{3}{c}{\expemba{}} & \multirow{2}{*}{\expembe{}} \\
{} & $D_{\text{model}} = 32$ & $D_{\text{model}} = 64$ & $D_{\text{model}} = 128$ & {} \\
\midrule
\textsc{Bool8} & 36.1 & 36.1 & 35.5 & 100.0 \\
\textsc{oneV-Poly13} & 38.0 & 38.5 & 38.1 & 99.9 \\
\textsc{SimpPoly10} & 37.7 & 32.2 & 34.5 & 99.9 \\
\textsc{SimpBool8} & 41.6 & 46.1 & 43.5 & 99.8 \\
\textsc{Bool10} & 9.6 & 9.8 & 8.9 & 94.0 \\
\textsc{SimpBool10} & 23.0 & 26.8 & 27.3 & 96.5 \\
\textsc{BoolL5} & 40.5 & 42.0 & 42.2 & 68.7 \\
\textsc{Poly8} & 34.9 & 35.0 & 34.5 & 95.2 \\
\textsc{SimpPoly8} & 38.5 & 36.7 & 36.1 & 99.6 \\
\textsc{SimpBoolL5} & 58.2 & 59.3 & 58.3 & 80.9 \\
\textsc{oneV-Poly10} & 36.1 & 39.6 & 37.9 & 77.3 \\
\textsc{Bool5} & 41.1 & 43.2 & 34.6 & 80.2 \\
\textsc{Poly5} & 14.7 & 8.2 & 4.2 & 46.2 \\
\textsc{SimpPoly5} & 30.9 & 32.6 & 42.2 & 47.9 \\
\bottomrule


### Compositionality Results

In [15]:
compositionality_datasets = [
    ("boolean5", "boolean10"),
    ("boolean5", "boolean8"),
    ("largeBoolean5", "boolean8"),
    ("oneVarPoly10", "oneVarPoly13"),
    ("poly5", "poly8"),
    ("poly5", "simplepoly10"),
    ("poly8", "oneVarPoly13"),
    ("poly8", "simplepoly10"),
    ("poly8", "simplepoly8"),
    ("simplepoly5", "simplepoly10"),
    ("simplepoly8", "simplepoly10"),
]

def read_compositionality_results(model, dataset):
    resultfilepattern = f"../models/semvec/equivexp/{model}/compositionality_{dataset}*"
    
    resultfilelist = glob.glob(resultfilepattern)
    assert len(resultfilelist) == 1, f"No file for model {model} and dataset {dataset}"
    resultfile = resultfilelist[0]
    
    results = TestingResults.load(resultfile)
    
    # Validate the test files
    assert results.args.test_file.endswith(f"{dataset}-neweqtestset.json.gz"), f"Invalid test file {results.args.test_file}"
    assert results.args.full_file.endswith(f"{dataset}.json.gz"), f"Invalid full file {results.args.full_file}"
    assert results.args.save_dir.endswith(model)
    
    return results.accuracy[0]["val/score@5/max"]

In [16]:
def print_compositionality_results():
    scores = []
    
    print("\\toprule")
    print("From $\\rightarrow$ To & $score_5(\%)$ \\\\")
    print("\\midrule")
    for elem in compositionality_datasets:
        model, dataset = elem[0], elem[1]
        score = float(read_compositionality_results(model, dataset))
        fromdisplayname = datasets[model]
        todisplayname = datasets[dataset]
        
        score = round(score * 100, 1)
        
        print(f"{fromdisplayname} $\\rightarrow$ {todisplayname} & {score} \\\\")
        
    print("\\bottomrule")
        
print_compositionality_results()

\toprule
From $\rightarrow$ To & $score_5(\%)$ \\
\midrule
\textsc{Bool5} $\rightarrow$ \textsc{Bool10} & 12.5 \\
\textsc{Bool5} $\rightarrow$ \textsc{Bool8} & 32.5 \\
\textsc{BoolL5} $\rightarrow$ \textsc{Bool8} & 29.3 \\
\textsc{oneV-Poly10} $\rightarrow$ \textsc{oneV-Poly13} & 29.8 \\
\textsc{Poly5} $\rightarrow$ \textsc{Poly8} & 32.9 \\
\textsc{Poly5} $\rightarrow$ \textsc{SimpPoly10} & 45.1 \\
\textsc{Poly8} $\rightarrow$ \textsc{oneV-Poly13} & 29.9 \\
\textsc{Poly8} $\rightarrow$ \textsc{SimpPoly10} & 58.9 \\
\textsc{Poly8} $\rightarrow$ \textsc{SimpPoly8} & 98.9 \\
\textsc{SimpPoly5} $\rightarrow$ \textsc{SimpPoly10} & 45.5 \\
\textsc{SimpPoly8} $\rightarrow$ \textsc{SimpPoly10} & 65.9 \\
\bottomrule


### Training Details

In [17]:
def read_semvec_epoch_details(filepath):
    output = {}
    with open(filepath, "r") as f:
        csvreader = csv.DictReader(f)
        for row in csvreader:
            details = {
                "min_epochs": int(row["Min Epochs"]),
                "max_epochs": int(row["Max Epochs"]),
                "patience": int(row["Wait Epochs"]),
            }
            output[row["Dataset"]] = details
            
    return output


def print_semvec_epochs_details_table():
    expembe = read_semvec_epoch_details("../models/semvec/SemVec_ExpEmb-E.csv")
    expemba = read_semvec_epoch_details("../models/semvec/SemVec_ExpEmb-A.csv")
    
    print("\\toprule")
    print("\\multirow{2}{*}{Dataset} & \multicolumn{3}{c}{\expemba{}} & \multicolumn{3}{c}{\expembe{}} \\\\")
    print("{} & Minimum & Maximum & Patience & Minimum & Maximum & Patience \\\\")
    print("\\midrule")
    for dataset, displayname in datasets.items():
        e_details = expembe[dataset]
        a_details = expemba[dataset]
        
        print(
            f"{displayname} & " + \
            f"{a_details['min_epochs']:,} & {a_details['max_epochs']:,} & {a_details['patience']:,} & " + \
            f"{e_details['min_epochs']:,} & {e_details['max_epochs']:,} & {e_details['patience']:,} \\\\"
        )
        
    print("\\bottomrule")
    
print_semvec_epochs_details_table()

\toprule
\multirow{2}{*}{Dataset} & \multicolumn{3}{c}{\expemba{}} & \multicolumn{3}{c}{\expembe{}} \\
{} & Minimum & Maximum & Patience & Minimum & Maximum & Patience \\
\midrule
\textsc{Bool8} & 175 & 3,485 & 70 & 2 & 32 & 2 \\
\textsc{oneV-Poly13} & 424 & 8,475 & 170 & 3 & 52 & 2 \\
\textsc{SimpPoly10} & 820 & 16,394 & 328 & 4 & 77 & 2 \\
\textsc{SimpBool8} & 1,163 & 23,256 & 466 & 6 & 116 & 3 \\
\textsc{Bool10} & 1,000 & 20,000 & 400 & 9 & 169 & 4 \\
\textsc{SimpBool10} & 1,924 & 38,462 & 770 & 18 & 354 & 8 \\
\textsc{BoolL5} & 1,087 & 21,740 & 435 & 47 & 926 & 19 \\
\textsc{Poly8} & 3,572 & 71,429 & 1,429 & 100 & 1,989 & 40 \\
\textsc{SimpPoly8} & 12,500 & 250,000 & 5,000 & 226 & 4,505 & 91 \\
\textsc{SimpBoolL5} & 4,167 & 83,334 & 1,667 & 382 & 7,634 & 153 \\
\textsc{oneV-Poly10} & 25,000 & 500,000 & 10,000 & 1,000 & 20,000 & 400 \\
\textsc{Bool5} & 25,000 & 500,000 & 10,000 & 1,389 & 27,778 & 556 \\
\textsc{Poly5} & 50,000 & 1,000,000 & 20,000 & 16,667 & 333,334 & 6,667 \\
\text

### ExpEmb-A vs ExpEmb-E (Same Dataset Size)

In [18]:
def read_expembe_small_results(dataset, autoencoder):
    if autoencoder:
        resultfilepattern = f"../models/semvec/autoencoder/h64/{dataset}/results-*.yaml"
    else:
        resultfilepattern = f"../models/semvec/equivexp_small/{dataset}/results-*.yaml"
    
    resultfilelist = glob.glob(resultfilepattern)
    assert len(resultfilelist) == 1
    resultfile = resultfilelist[0]
    
    results = TestingResults.load(resultfile)
    
    # Validate the test files
    assert results.args.test_file.endswith(f"{dataset}-neweqtestset.json.gz"), f"Invalid test file {results.args.test_file}"
    assert results.args.full_file.endswith(f"{dataset}.json.gz"), f"Invalid full file {results.args.full_file}"
    
    return results.accuracy[0]["val/score@5/max"]


def print_expembe_small_results_table():
    scores = []

    for dataset, displayname in datasets.items():
        expembe_score = float(read_expembe_small_results(dataset, autoencoder = False))
        expemba_score = float(read_expembe_small_results(dataset, autoencoder = True))
        
        expembe_score = round(expembe_score * 100, 1)
        expemba_score = round(expemba_score * 100, 1)

        scores.append({
            "dataset": displayname,
            "expemba": expemba_score,
            "expembe": expembe_score,
        })

    print("\\toprule")
    print("Dataset & \\expemba{} & \\expembe{} \\\\")
    print("\\midrule")
    for score in scores:
        print(f"{score['dataset']} & {score['expemba']} & {score['expembe']} \\\\")
        
    print(f"\\bottomrule")
    
    
print_expembe_small_results_table()

\toprule
Dataset & \expemba{} & \expembe{} \\
\midrule
\textsc{Bool8} & 30.6 & 99.8 \\
\textsc{oneV-Poly13} & 38.7 & 97.6 \\
\textsc{SimpPoly10} & 40.1 & 90.7 \\
\textsc{SimpBool8} & 36.9 & 76.5 \\
\textsc{Bool10} & 10.8 & 20.8 \\
\textsc{SimpBool10} & 24.4 & 41.4 \\
\textsc{BoolL5} & 38.3 & 60.4 \\
\textsc{Poly8} & 32.7 & 31.6 \\
\textsc{SimpPoly8} & 47.6 & 48.6 \\
\textsc{SimpBoolL5} & 55.1 & 77.1 \\
\textsc{oneV-Poly10} & 59.8 & 61.4 \\
\textsc{Bool5} & 36.4 & 36.7 \\
\textsc{Poly5} & 5.7 & 27.3 \\
\textsc{SimpPoly5} & 28.1 & 19.8 \\
\bottomrule
